# 1. PCA Programming

## 1.1. Download Dataset

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import seaborn as sns
from PCA import *
from Utils.Utils import *
import warnings

warnings.filterwarnings("ignore")

iris = load_iris()
X = pd.DataFrame(iris['data'], columns = iris['feature_names'])
y = pd.DataFrame(iris['target'], columns = ['species'])

## 1.2. EDA

In [ ]:
X.head()

In [ ]:
X.info()

In [ ]:
X.describe()

In [ ]:
X_all = pd.concat([X, y], axis = 1)

sns.pairplot(X_all, hue='species', palette='tab10', diag_kind='hist')

# 2. PCA

In [ ]:
X, y = np.array(X), np.array(y)

In [ ]:
my_pca = MyPCA(n_components = 2)
my_pca.fit(X)

In [ ]:
# Transform
X_transformed = my_pca.transform(X)
print("\nDữ liệu sau khi biến đổi (X_transformed) có shape:", X_transformed.shape)
print("Một vài mẫu dữ liệu sau khi biến đổi:")
print(X_transformed[:5, :])

# Check PCA của sklearn

In [ ]:
from sklearn.decomposition import PCA as SklearnPCA

print("--- Kiểm tra với sklearn.decomposition.PCA ---")
# 1. Khởi tạo PCA của sklearn
sklearn_pca = SklearnPCA(n_components=2)

# 2. Khớp mô hình với dữ liệu X
sklearn_pca.fit(X)

# 3. Biến đổi dữ liệu X
X_transformed_sklearn = sklearn_pca.transform(X)

print("\nTỷ lệ phương sai giải thích (EVR) cho từng thành phần (sklearn.explained_variance_ratio_):")
# Đây là EVR cho n_components đã chọn
print(sklearn_pca.explained_variance_ratio_)
# Tương đương với self.explained_variance_ratio_[:n_components] của MyPCA

print("\nPhương sai giải thích bởi từng thành phần (sklearn.explained_variance_):")
# Đây là các trị riêng tương ứng với n_components đã chọn, không phải tỷ lệ
print(sklearn_pca.explained_variance_)
# Tương đương với self.explained_variance_[:n_components] của MyPCA

# sklearn không có thuộc tính CEVR trực tiếp, nhưng có thể dễ dàng tính từ EVR
cumulative_explained_variance_ratio_sklearn = np.cumsum(sklearn_pca.explained_variance_ratio_)
print("\nTỷ lệ phương sai giải thích tích lũy (CEVR) tính từ sklearn.explained_variance_ratio_:")
for i, cum_ratio in enumerate(cumulative_explained_variance_ratio_sklearn):
    print(f"  PC1 đến PC{i+1}: {cum_ratio:.4f}")
print(f"Tổng phương sai được giải thích bởi {sklearn_pca.n_components_} thành phần: {cumulative_explained_variance_ratio_sklearn[-1]:.4f}")
# Giá trị cuối cùng này tương đương với self.cumulative_explained_variance_ratio_[n_components-1] của MyPCA

print("\nDữ liệu sau khi biến đổi (X_transformed_sklearn) có shape:", X_transformed_sklearn.shape)
print("Một vài mẫu dữ liệu sau khi biến đổi (sklearn):")
print(X_transformed_sklearn[:5, :]) # Tương đương với X_transformed của MyPCA
print("---------------------------------------------")

# 3. K-Means Clustering

## 3.1. Data Preprocessing

In [ ]:
X = pd.read_csv('Dataset/ABIDE2.csv', index_col = 0)
y = X['group']

In [ ]:
X.head(5)

In [ ]:
X.info()

In [ ]:
X.describe()

In [ ]:
run_experiment(
    "Dataset/ABIDE2.csv",
    skew_threshold=0.1,
    component_range=[1],
    k=2,
    visualize=False,
    algorithm="KMeans",
    use_poly=False
)

In [ ]:
run_experiment(
    "Dataset/ABIDE2.csv",
    skew_threshold=0.1,
    component_range=[2],
    k=2,
    visualize=False,
    algorithm="KMeans",
    use_poly=True
)

In [ ]:
run_experiment(
    "Dataset/ABIDE2.csv",
    skew_threshold=1.0,
    component_range=[10],
    k=2,
    visualize=False,
    algorithm="GMM",
    use_poly=False
)

In [ ]:
run_experiment(
    "Dataset/ABIDE2.csv",
    skew_threshold=0.1,
    component_range=[5],
    k=2,
    visualize=False,
    algorithm="GMM",
    use_poly=True
)